In [ ]:
import 서울시_지하철_승하차_현황_API as sw_pa
import 서울시유무임승차현황 as sw_py_fr
import pandas as pd
from datetime import datetime, timedelta

In [ ]:
def Data_input_all():
    ### 전체 데이터 가져오기
    sDt="20150101"
    eDt=datetime.today().strftime('%Y%m%d')
    ### 지하철 승하차인원 데이터 조회 및 저장
    df0=sw_pa.main_api(sDt, eDt)
    df0.to_csv('c:/pydata/seoul_sw_inout.csv', index=False)
    ### 지하철 유/무임승하차인원 데이터 조회 및 저장
    df1=sw_py_fr.main_api(sDt[:-2], eDt[:-2])
    df1.to_csv('c:/pydata/seoul_sw_payfree.csv', index=False)
    print("=====API 데이터 가져오기 성공=====")

In [ ]:
# Data_input_all()

## 서울시 지하철 탑승현황
- seoul_sw_inout.csv / seoul_sw_payfree.csv 이용
- 각 파일에서 추가되지 않은 데이터를 API를 이용해 가져와 추가하기
- 승하차 데이터 / 유임/무임 승하차 데이터를 이용해 년도별/월별/데이터 생성
- 역 입력시 해당 역의 승하차, 유/무임 데이터 확인 및 시각화
- 유임/무임 승하차가 가장 많은 역 확인

### 데이터 읽어오기 및 데이터 추가
- 두 데이터 가져오기

In [ ]:
df0=pd.read_csv('c:/pydata/seoul_sw_inout.csv')
df1=pd.read_csv('c:/pydata/seoul_sw_payfree.csv')

- 오늘 날짜를 기준으로 없는 데이터 추가하기
 1. 읽어온 데이터의 마지막 데이터 날짜/년월을 가져온다.
 2. 마지막 날짜가 오늘날짜/오늘년월 일치/불일치 확인(일치=종료, 불일치=API로 부터 가져오기)
 3. 가져온 데이터 추가 후 pydata 업데이트

- 일자별 승하차 데이터 추가

In [ ]:
if str(df0.iloc[-1,0])!=datetime.today().strftime('%Y%m%d'):
    sDt=datetime.strptime(str(df0.iloc[-1,0]),'%Y%m%d')+timedelta(1)
    sDt=sDt.strftime('%Y%m%d')
    eDt=datetime.today().strftime('%Y%m%d')

    df=sw_pa.main_api(sDt, eDt)
    
    df0=pd.concat([df0, df], ignore_index=True)
    df0.to_csv('c:/pydata/seoul_sw_inout.csv', index=False)


- 년월별 유임/무임 승하차 데이터 업데이트

In [ ]:
sDt=datetime.strptime(str(df1.iloc[-1,0])+"27",'%Y%m%d')+timedelta(5)
if sDt.strftime('%Y%m')!=datetime.today().strftime('%Y%m'):
    sDt=sDt.strftime('%Y%m')
    eDt=datetime.today().strftime('%Y%m')
    df=sw_py_fr.main_api(sDt, eDt)
    df1=pd.concat([df0, df], ignore_index=True)
    df1.to_csv('c:/pydata/seoul_sw_payfree.csv', index=False)

print("유/무임 데이터 업데이트 완료")

In [ ]:
print(df0.info())
print(df1.info())

- 승하차 데이터/ 유임/무임 승하차 데이터를 이용해 년도별/월별 데이터 생성

In [ ]:
df0=df0.astype({'사용일':'str'})
print(df0.info())
df1=df1.rename(columns={'사용일':'사용년월'})
df1=df1.astype({'사용년월':'str'})
print(df1.info())

In [ ]:
df0['사용년월']=df0['사용일'].str[:6] #필드 추가
df2=df0.groupby(['사용년월','라인','역명']).sum() # 구글 합계 df2 저장
df2=df2.reset_index() #df2 index 재 설정후 df2 업데이트
df1.head()

In [150]:
for i in df1.index:
    print(df2[(df2['사용년월']==df1.loc[i][0])&(df2['라인']==df1.loc[i][1])&(df2['역명']==df1.loc[i][2])])
  


       사용년월   라인   역명      승차      하차
105  201501  4호선  삼각지  175501  220537
       사용년월   라인   역명      승차      하차
111  201501  4호선  신용산  408120  435175
       사용년월   라인  역명      승차      하차
113  201501  4호선  이촌  307143  337390
       사용년월   라인  역명     승차     하차
100  201501  4호선  동작  66583  68255
       사용년월   라인     역명      승차      하차
115  201501  4호선  총신대입구  705946  783199
       사용년월   라인  역명       승차      하차
104  201501  4호선  사당  1014983  820043
      사용년월   라인   역명     승차     하차
95  201501  4호선  남태령  32936  27184
       사용년월   라인  역명      승차      하차
144  201501  5호선  방화  228133  217578
       사용년월   라인   역명      승차      하차
122  201501  5호선  개화산  182882  167157
       사용년월   라인    역명      승차      하차
131  201501  5호선  김포공항  255026  227276
       사용년월   라인  역명      승차      하차
147  201501  5호선  송정  322701  317092
       사용년월   라인  역명     승차     하차
136  201501  5호선  마곡  92548  77249
       사용년월   라인  역명      승차      하차
142  201501  5호선  발산  323376  324484
       사용년월   라인   역명      승차   

       사용년월   라인   역명      승차      하차
234  201501  7호선  사가정  514310  497512
       사용년월   라인   역명      승차      하차
246  201501  7호선  용마산  190879  179941
       사용년월   라인  역명      승차      하차
251  201501  7호선  중곡  311985  287921
       사용년월   라인  역명      승차      하차
215  201501  7호선  군자  460323  347638
       사용년월   라인      역명      승차      하차
244  201501  7호선  어린이대공원  418429  410742
       사용년월   라인    역명      승차      하차
211  201501  7호선  건대입구  551820  580401
       사용년월   라인     역명      승차      하차
225  201501  7호선  뚝섬유원지  244315  233831
       사용년월   라인  역명      승차      하차
255  201501  7호선  청담  727656  823501
       사용년월   라인    역명      승차      하차
210  201501  7호선  강남구청  554063  599188
       사용년월   라인  역명      승차      하차
259  201501  7호선  학동  742550  813745
       사용년월   라인  역명      승차      하차
222  201501  7호선  논현  633851  699799
       사용년월   라인  역명      승차      하차
229  201501  7호선  반포  196919  185920
       사용년월   라인     역명      승차      하차
212  201501  7호선  고속터미널  677123  585941
      

      사용년월   라인  역명      승차      하차
29  201501  2호선  서초  690565  705721
      사용년월   라인  역명      승차      하차
23  201501  2호선  방배  610840  616838
      사용년월   라인  역명       승차       하차
25  201501  2호선  사당  1385747  1525548
      사용년월   라인   역명      승차      하차
16  201501  2호선  낙성대  942788  906543
      사용년월   라인     역명       승차       하차
28  201501  2호선  서울대입구  1646581  1620564
      사용년월   라인  역명      승차      하차
24  201501  2호선  봉천  759890  707083
      사용년월   라인  역명       승차       하차
37  201501  2호선  신림  2278447  2236704
      사용년월   라인   역명      승차      하차
35  201501  2호선  신대방  844486  811582
      사용년월   라인       역명       승차       하차
14  201501  2호선  구로디지털단지  1984765  1998025
      사용년월   라인  역명      승차       하차
18  201501  2호선  대림  984518  1023860
      사용년월   라인   역명       승차       하차
36  201501  2호선  신도림  2044253  2022323
      사용년월   라인  역명      승차      하차
22  201501  2호선  문래  585927  590420
      사용년월   라인     역명      승차      하차
45  201501  2호선  영등포구청  671143  666855
      사용년월   라

       사용년월   라인   역명      승차      하차
833  201502  9호선  노량진  817376  864768
       사용년월   라인  역명      승차      하차
832  201502  9호선  노들  126944  106065
       사용년월   라인         역명      승차      하차
849  201502  9호선  흑석(중앙대입구)  265374  273156
       사용년월   라인  역명     승차     하차
835  201502  9호선  동작  46872  33976
       사용년월   라인   역명     승차     하차
829  201502  9호선  구반포  87859  93702
       사용년월   라인   역명     승차     하차
843  201502  9호선  신반포  78207  78892
       사용년월   라인     역명      승차      하차
827  201502  9호선  고속터미널  405150  453218
       사용년월   라인  역명     승차     하차
838  201502  9호선  사평  73253  69433
       사용년월   라인   역명      승차      하차
841  201502  9호선  신논현  805680  788307
       사용년월   라인   역명       승차       하차
550  201502  1호선  서울역  1916627  1701043
       사용년월   라인  역명      승차      하차
551  201502  1호선  시청  613510  608754
       사용년월   라인  역명       승차       하차
554  201502  1호선  종각  1271236  1199408
       사용년월   라인    역명       승차      하차
555  201502  1호선  종로3가  1026335  962203
       사용년

       사용년월   라인     역명      승차      하차
665  201502  4호선  한성대입구  409167  394887
       사용년월   라인  역명       승차       하차
666  201502  4호선  혜화  1148886  1203535
       사용년월   라인   역명      승차      하차
646  201502  4호선  동대문  633854  654328
       사용년월   라인         역명      승차      하차
647  201502  4호선  동대문역사문화공원  609597  550912
       사용년월   라인   역명      승차      하차
664  201502  4호선  충무로  845803  876055
       사용년월   라인  역명       승차       하차
649  201502  4호선  명동  1159858  1268388
       사용년월   라인  역명      승차      하차
667  201502  4호선  회현  742490  784598
       사용년월   라인   역명      승차      하차
655  201502  4호선  서울역  271083  504762
       사용년월   라인    역명      승차      하차
658  201502  4호선  숙대입구  425278  415581
       사용년월   라인   역명      승차      하차
653  201502  4호선  삼각지  145411  181489
       사용년월   라인   역명      승차      하차
659  201502  4호선  신용산  349149  375004
       사용년월   라인  역명      승차      하차
661  201502  4호선  이촌  246647  272590
       사용년월   라인  역명     승차     하차
648  201502  4호선  동작  58559  61246


       사용년월   라인  역명      승차      하차
800  201502  7호선  중화  317625  301658
       사용년월   라인  역명      승차      하차
786  201502  7호선  상봉  476177  444869
       사용년월   라인  역명      승차      하차
776  201502  7호선  면목  398346  383750
       사용년월   라인   역명      승차      하차
782  201502  7호선  사가정  443855  430070
       사용년월   라인   역명      승차      하차
794  201502  7호선  용마산  163826  156221
       사용년월   라인  역명      승차      하차
799  201502  7호선  중곡  271333  251615
       사용년월   라인  역명      승차      하차
763  201502  7호선  군자  390632  292658
       사용년월   라인      역명      승차      하차
792  201502  7호선  어린이대공원  350261  344221
       사용년월   라인    역명      승차      하차
759  201502  7호선  건대입구  466358  494889
       사용년월   라인     역명      승차      하차
773  201502  7호선  뚝섬유원지  207947  199846
       사용년월   라인  역명      승차      하차
803  201502  7호선  청담  613097  693003
       사용년월   라인    역명      승차      하차
758  201502  7호선  강남구청  459325  496162
       사용년월   라인  역명      승차      하차
807  201502  7호선  학동  615361  672132
       사용년월 

        사용년월   라인  역명      승차      하차
1293  201503  6호선  월곡  462780  436823
        사용년월   라인   역명      승차      하차
1285  201503  6호선  상월곡  221710  208467
        사용년월   라인   역명      승차      하차
1274  201503  6호선  돌곶이  323899  295028
        사용년월   라인  역명      승차      하차
1287  201503  6호선  석계  461178  464147
        사용년월   라인    역명      승차      하차
1300  201503  6호선  태릉입구  174784  208761
        사용년월   라인   역명      승차      하차
1303  201503  6호선  화랑대  450232  346152
        사용년월   라인          역명      승차      하차
1281  201503  6호선  봉화산(서울의료원)  310440  291305
        사용년월   라인  역명     승차     하차
1345  201503  7호선  장암  82616  30357
        사용년월   라인   역명      승차      하차
1320  201503  7호선  도봉산  403464  383365
        사용년월   라인   역명      승차      하차
1335  201503  7호선  수락산  496131  477472
        사용년월   라인  역명      승차      하차
1322  201503  7호선  마들  404708  367861
        사용년월   라인  역명      승차      하차
1317  201503  7호선  노원  704681  763544
        사용년월   라인  역명      승차      하차
1346  201503  7호선  중계  5

KeyboardInterrupt: 

In [151]:
df0['사용년월']=df0['사용일'].str[:6] #필드 추가

df2=df0.groupby(['사용년월','라인','역명']).sum() # 구글 합계 df2 저장
df3=df1.groupby(['사용년월','라인','역명']).sum()


In [152]:
df2

승차       하차
사용년월   라인  역명                   
201501 1호선 동대문   483789   542499
           동묘앞   285550   300658
           서울역  2160616  1916768
           시청    748569   742981
           신설동   512694   495320
...                 ...      ...
202107 중앙선 원덕      1738     1723
           중랑     24224    23085
           지평       348      378
           팔당      4966     4910
           회기    101309    97726

[45944 rows x 2 columns]

In [153]:
df3

유임승차     무임승차    유임하차    무임하차
사용년월   라인  역명                                   
201501 1호선 동대문   336981   390926  132944  137814
           동묘앞   155556   171171  124838  124116
           서울역  1890411  1667163  238403  220008
           시청    639441   636014   87392   85689
           신설동   374981   363032  124078  118929
...                 ...      ...     ...     ...
202106 중앙선 원덕      5259     6214    5030    5980
           중랑    110574    43835  104494   42601
           지평       872     1110    1152    1077
           팔당     22609     8316   23186    8452
           회기    547246   109130  526911  106975

[40866 rows x 4 columns]

In [165]:
df4=pd.concat([df2,df3], axis=1, join='inner') #join='outer' => 합집합, join='inner'=> 교집합
df4=df4.reset_index()
df4

,사용년월,라인,역명,승차,하차,유임승차,무임승차,유임하차,무임하차
0,201501,1호선,동대문,483789,542499,336981,390926,132944,137814
1,201501,1호선,동묘앞,285550,300658,155556,171171,124838,124116
2,201501,1호선,서울역,2160616,1916768,1890411,1667163,238403,220008
3,201501,1호선,시청,748569,742981,639441,636014,87392,85689
4,201501,1호선,신설동,512694,495320,374981,363032,124078,118929
...,...,...,...,...,...,...,...,...,...
40839,202106,중앙선,원덕,11473,11010,5259,6214,5030,5980
40840,202106,중앙선,중랑,154409,147095,110574,43835,104494,42601
40841,202106,중앙선,지평,1982,2229,872,1110,1152,1077
40842,202106,중앙선,팔당,30925,31638,22609,8316,23186,8452


In [171]:
df4[df4['사용년월']=='202105']['유임승차'].max()

1967326

In [174]:
df4[(df4['사용년월']=='202105')&(df4['무임승차']==df4[df4['사용년월']=='202105']['무임승차'].max())]

,사용년월,라인,역명,승차,하차,유임승차,무임승차,유임하차,무임하차
39635,202105,1호선,종로3가,673327,617934,421121,252206,380881,237053


In [176]:
df4[(df4['사용년월']=='202106')&(df4['무임하차']==df4[df4['사용년월']=='202106']['무임하차'].max())]

,사용년월,라인,역명,승차,하차,유임승차,무임승차,유임하차,무임하차
40241,202106,1호선,제기동,459463,470591,224878,234585,218527,252064


In [178]:
df4[df4['사용년월']=='202106'].sort_values(by='무임승차', ascending=False).head()

,사용년월,라인,역명,승차,하차,유임승차,무임승차,유임하차,무임하차
40243,202106,1호선,종로3가,684833,625095,427162,257671,383856,241239
40244,202106,1호선,종로5가,598069,597269,358583,239486,366760,230509
40245,202106,1호선,청량리(서울시립대입구),553525,555155,317436,236089,314995,240160
40241,202106,1호선,제기동,459463,470591,224878,234585,218527,252064
40319,202106,3호선,연신내,1014329,953790,786054,228275,732219,221571


In [180]:
df4[df4['사용년월']=='202106'].sort_values(by='무임하차', ascending=False).head()

,사용년월,라인,역명,승차,하차,유임승차,무임승차,유임하차,무임하차
40241,202106,1호선,제기동,459463,470591,224878,234585,218527,252064
40243,202106,1호선,종로3가,684833,625095,427162,257671,383856,241239
40245,202106,1호선,청량리(서울시립대입구),553525,555155,317436,236089,314995,240160
40244,202106,1호선,종로5가,598069,597269,358583,239486,366760,230509
40319,202106,3호선,연신내,1014329,953790,786054,228275,732219,221571


In [179]:
df4[df4['사용년월']=='202106'].sort_values(by='무임승차', ascending=True).head()

,사용년월,라인,역명,승차,하차,유임승차,무임승차,유임하차,무임하차
40645,202106,경의선,김포공항,1,0,1,0,0,0
40433,202106,6호선,신내,7,0,7,0,0,0
40640,202106,경의선,계양,3,0,3,0,0,0
40815,202106,일산선,지축,5,0,5,0,0,0
40639,202106,경의선,검암,1,0,1,0,0,0
